In [1]:
import numpy as np
import os
import sys

from tinytorch.core.layers import Linear, Dropout, Sequential
from tinytorch.core.tensor import Tensor
from tinytorch.core.activations import ReLU, Sigmoid
from tinytorch.core.autograd import enable_autograd

enable_autograd()

## Unit Test: Linear Layers

In [2]:
XAVIER_SCALE_FACTOR = 2.0

def test_unit_linear_layer():
    print("🔬 Unit Test: Linear Layer...")
    layer = Linear(784, 256)

    # Test layer creation
    assert layer.in_features == 784
    assert layer.out_features == 256
    assert layer.weight.shape == (784, 256)
    assert layer.bias.shape == (256,)

    # Test xavier initialization (weights should be reasonably scaled)
    weight_std = np.std(layer.weight.data)
    expected_std = np.sqrt(XAVIER_SCALE_FACTOR / 784)
    assert 0.5 * expected_std < weight_std < 2.0 * expected_std, f'Weight std {weight_std} not close to Xavier {expected_std} '

    # Test bias initialization  (should be zeros)
    assert np.allclose(layer.bias.data, 0), 'Bias should be initialized to zeros'

    # Test forward pass
    x = Tensor(np.random.randn(32, 784)) # batch of 32 samples
    y = layer.forward(x)
    assert y.shape == (32, 256), f'Expected shapr (32, 256, got {y.shape}'
    
    # Test no bias option
    layer_no_bias = Linear(10, 5, bias= False)
    assert layer_no_bias.bias is None
    params = layer_no_bias.parameters()
    assert len(params) == 1 

    # Test parameters method
    params = layer.parameters()
    assert len(params) == 2
    assert params[0] is layer.weight
    assert params[1] is layer.bias


    print("✅ Linear layer works correctly!")

if __name__ == '__main__':
    test_unit_linear_layer()

🔬 Unit Test: Linear Layer...
✅ Linear layer works correctly!


## Edge case test: Linear 

In [3]:
def test_edge_cases_linear():
    print("🔬 Edge Case Tests: Linear Layer...")

    layer = Linear(10, 5)

    # Test single sample (should handle 2D input)
    x_2d = Tensor(np.random.randn(1, 10))
    y = layer.forward(x_2d)
    assert y.shape == (1, 5), 'should handle single cases'

    # Test zero batch size (edge case)
    x_empty = Tensor(np.random.randn(0, 10))
    y_empty = layer.forward(x_empty)
    assert y_empty.shape == (0, 5), 'should handle empty batch'

    # Test numerical stability with large weights
    layer_large = Linear(10, 5)
    layer_large.weight.data = np.ones((10, 5)) * 100
    x = Tensor(np.ones((1, 10)))
    y = layer_large.forward(x)
    assert not np.any(np.isnan(y.data)), 'should not produce NaN with large weights'
    assert not np.any(np.isinf(y.data)), 'should not produce inf with large weights'

    # Test with no bias
    layer_no_bias= Linear(10, 5, bias= False)
    x = Tensor(np.random.randn(4, 10))
    y = layer_no_bias.forward(x)
    assert y.shape == (4, 5), 'should work without bias'

    print("✅ Edge cases handled correctly!")
if __name__ =='__main__':
    test_edge_cases_linear()

🔬 Edge Case Tests: Linear Layer...
✅ Edge cases handled correctly!


## Parameter Collections Tests: Linear

In [4]:
def test_parameter_collection_linear():
    print("🔬 Parameter Collection Test: Linear Layer...")

    layer = Linear(10, 5)

    # verify parameter collection works
    params = layer.parameters()
    assert len(params) == 2, 'should return 2 parameters (weight and bias)'
    assert params[0].shape == (10, 5), 'first param should be weight'
    assert params[1].shape == (5,), 'second parameter should be bias'

    # Test layer without bias
    layer_no_bias = Linear(10, 5, bias= False)
    params_no_bias = layer_no_bias.parameters()
    assert len(params_no_bias) == 1, 'should return 1 parameter (weight only)'
    
    print("✅ Parameter collection works correctly!")

if __name__ == '__main__':
    test_parameter_collection_linear()

🔬 Parameter Collection Test: Linear Layer...
✅ Parameter collection works correctly!


## Unit test: Dropout Layer

In [5]:
def test_unit_dropout_layer():
    print("🔬 Unit Test: Dropout Layer...")

    dropout = Dropout(0.5)
    assert dropout.p == 0.5

    # test inference mode, should pass unchanged
    x = Tensor([1, 2, 3, 4])
    y_inference = dropout.forward(x, training= False)
    assert np.array_equal(x.data, y_inference.data), 'Inference should pass through unchanged'

    # Test training mode with zero dropout (should pass through unchanged)
    dropout_zero = Dropout(0.0)
    y_zero = dropout_zero.forward(x, training= True)
    assert np.array_equal(x.data, y_zero.data), 'Zero dropout should pass through unchanged'

    dropout_full = Dropout(1.0)
    y_full = dropout_full.forward(x, training= True)
    assert np.allclose(y_full.data, 0), 'Full dropout should zero everything'

    # Test training mode with partial dropout
    np.random.seed(42)
    x_large = Tensor(np.ones((1000,)))
    y_train = dropout.forward(x_large, training= True)

    # count non zero elements
    non_zero_count = np.count_nonzero(y_train.data)
    expected = 500
    std_error = np.sqrt(1000 * 0.5 * 0.5)
    lower_bound = expected - 3 * std_error
    upper_bound = expected + 3 * std_error
    assert lower_bound < non_zero_count < upper_bound, f'Expected {expected}±{3*std-error:.0f} survivors, got {non_zero_count}'

    # Test sclaing (surviving element should be scaled y 1 / (1- p) = 2.0)
    surviving_values = y_train.data[y_train.data != 0]
    expected_value = 2.0 
    assert np.allclose(surviving_values, expected_value), f'Surviving values should be {expected_value}'

    # Test no parameters
    params = dropout.parameters()
    assert len(params) == 0, 'Dropout should have no parameters'

    # Test invalid probabilities
    try:
        Dropout(-0.1)
        assert False, 'Should raise ValueError for negative probability'
    except ValueError:
        pass

    try:
        Dropout(1.1)
        assert False, 'Should raise ValueEror for probability > 1'
    except ValueError:
        pass

    print("✅ Dropout layer works correctly!")

if __name__=='__main__':
    test_unit_dropout_layer()

🔬 Unit Test: Dropout Layer...
✅ Dropout layer works correctly!


## Integration Bringing it Together 

In [6]:
def analyze_layer_memory():
    """📊 Analyze memory usage patterns in layer operations."""
    print("📊 Analyzing Layer Memory Usage...")

    # Test different layer sizes
    layer_configs = [
        (784, 256),   # MNIST → hidden
        (256, 256),   # Hidden → hidden
        (256, 10),    # Hidden → output
        (2048, 2048), # Large hidden
    ]

    print("\nLinear Layer Memory Analysis:")
    print("Configuration → Weight Memory → Bias Memory → Total Memory")

    for in_feat, out_feat in layer_configs:
        # Calculate memory usage
        weight_memory = in_feat * out_feat * 4  # 4 bytes per float32
        bias_memory = out_feat * 4
        total_memory = weight_memory + bias_memory

        print(f"({in_feat:4d}, {out_feat:4d}) → {weight_memory/1024:7.1f} KB → {bias_memory/1024:6.1f} KB → {total_memory/1024:7.1f} KB")

    # Analyze multi-layer memory scaling
    print("\n💡 Multi-layer Model Memory Scaling:")
    hidden_sizes = [128, 256, 512, 1024, 2048]

    for hidden_size in hidden_sizes:
        # 3-layer MLP: 784 → hidden → hidden/2 → 10
        layer1_params = 784 * hidden_size + hidden_size
        layer2_params = hidden_size * (hidden_size // 2) + (hidden_size // 2)
        layer3_params = (hidden_size // 2) * 10 + 10

        total_params = layer1_params + layer2_params + layer3_params
        memory_mb = total_params * 4 / (1024 * 1024)

        print(f"Hidden={hidden_size:4d}: {total_params:7,} params = {memory_mb:5.1f} MB")

# Analysis will be run in main block

if __name__ == '__main__':
    analyze_layer_memory()

📊 Analyzing Layer Memory Usage...

Linear Layer Memory Analysis:
Configuration → Weight Memory → Bias Memory → Total Memory
( 784,  256) →   784.0 KB →    1.0 KB →   785.0 KB
( 256,  256) →   256.0 KB →    1.0 KB →   257.0 KB
( 256,   10) →    10.0 KB →    0.0 KB →    10.0 KB
(2048, 2048) → 16384.0 KB →    8.0 KB → 16392.0 KB

💡 Multi-layer Model Memory Scaling:
Hidden= 128: 109,386 params =   0.4 MB
Hidden= 256: 235,146 params =   0.9 MB
Hidden= 512: 535,818 params =   2.0 MB
Hidden=1024: 1,333,770 params =   5.1 MB
Hidden=2048: 3,716,106 params =  14.2 MB


In [7]:
def analyze_layer_performance():
    """📊 Analyze computational complexity of layer operations."""
    import time

    print("📊 Analyzing Layer Computational Complexity...")

    # Test forward pass FLOPs
    batch_sizes = [1, 32, 128, 512]
    layer = Linear(784, 256)

    print("\nLinear Layer FLOPs Analysis:")
    print("Batch Size → Matrix Multiply FLOPs → Bias Add FLOPs → Total FLOPs")

    for batch_size in batch_sizes:
        # Matrix multiplication: (batch, in) @ (in, out) = batch * in * out FLOPs
        matmul_flops = batch_size * 784 * 256
        # Bias addition: batch * out FLOPs
        bias_flops = batch_size * 256
        total_flops = matmul_flops + bias_flops

        print(f"{batch_size:10d} → {matmul_flops:15,} → {bias_flops:13,} → {total_flops:11,}")

    # Add timing measurements
    print("\nLinear Layer Timing Analysis:")
    print("Batch Size → Time (ms) → Throughput (samples/sec)")

    for batch_size in batch_sizes:
        x = Tensor(np.random.randn(batch_size, 784))

        # Warm up
        for _ in range(10):
            _ = layer.forward(x)

        # Time multiple iterations
        iterations = 100
        start = time.perf_counter()
        for _ in range(iterations):
            _ = layer.forward(x)
        elapsed = time.perf_counter() - start

        time_per_forward = (elapsed / iterations) * 1000  # Convert to ms
        throughput = (batch_size * iterations) / elapsed

        print(f"{batch_size:10d} → {time_per_forward:8.3f} ms → {throughput:12,.0f} samples/sec")

    print("\n💡 Key Insights:")
    print("🚀 Linear layer complexity: O(batch_size × in_features × out_features)")
    print("🚀 Memory grows linearly with batch size, quadratically with layer width")
    print("🚀 Dropout adds minimal computational overhead (element-wise operations)")
    print("🚀 Larger batches amortize overhead, improving throughput efficiency")

# Analysis will be run in main block
if __name__=='__main__':
    analyze_layer_performance()

📊 Analyzing Layer Computational Complexity...

Linear Layer FLOPs Analysis:
Batch Size → Matrix Multiply FLOPs → Bias Add FLOPs → Total FLOPs
         1 →         200,704 →           256 →     200,960
        32 →       6,422,528 →         8,192 →   6,430,720
       128 →      25,690,112 →        32,768 →  25,722,880
       512 →     102,760,448 →       131,072 → 102,891,520

Linear Layer Timing Analysis:
Batch Size → Time (ms) → Throughput (samples/sec)
         1 →    0.962 ms →        1,040 samples/sec
        32 →   31.517 ms →        1,015 samples/sec
       128 →  101.279 ms →        1,264 samples/sec
       512 →  404.145 ms →        1,267 samples/sec

💡 Key Insights:
🚀 Linear layer complexity: O(batch_size × in_features × out_features)
🚀 Memory grows linearly with batch size, quadratically with layer width
🚀 Dropout adds minimal computational overhead (element-wise operations)
🚀 Larger batches amortize overhead, improving throughput efficiency


In [8]:
model = Sequential(
    Linear(784, 256),
    ReLU(),
    Dropout(0.5),
    Linear(256, 128),
    ReLU(),
    Dropout(0.3),
    Linear(128, 10), 
    Sigmoid()
)
x = Tensor(np.random.random((30, 784)))
output = model(x)

## Module Integration Testing

In [9]:
def test_module():
    print("🧪 RUNNING MODULE INTEGRATION TEST")
    print("=" * 50)

    print("✅ Multi-layer network integration works!")
    print("Running unit tests...")
    test_unit_linear_layer()
    test_edge_cases_linear()
    test_parameter_collection_linear()
    test_unit_dropout_layer()

    print("\nRunning integration scenarios...")

    # Test realistic neural network construction with manual composition
    print("🔬 Integration Test: Multi-layer Network...")

    ReLU_class = ReLU
    # Build individual layers for manual composition
    layer1 = Linear(784, 128)
    activation1 = ReLU_class()
    dropout1 = Dropout(0.5)
    layer2 = Linear(128, 64)
    activation2 = ReLU_class()
    dropout2 = Dropout(0.3)
    layer3 = Linear(64, 10)

    # Test end-to-end forward pass with manual composition
    batch_size = 16
    x = Tensor(np.random.randn(batch_size, 784))

    # Manual forward pass
    x = layer1.forward(x)
    x = activation1.forward(x)
    x = dropout1.forward(x)
    x = layer2.forward(x)
    x = activation2.forward(x)
    x = dropout2.forward(x)
    output = layer3.forward(x)

    assert output.shape == (batch_size, 10), f"Expected output shape ({batch_size}, 10), got {output.shape}"

    # Test parameter counting from individual layers
    all_params = layer1.parameters() + layer2.parameters() + layer3.parameters()
    expected_params = 6  # 3 weights + 3 biases from 3 Linear layers
    assert len(all_params) == expected_params, f"Expected {expected_params} parameters, got {len(all_params)}"

    # Test individual layer functionality
    test_x = Tensor(np.random.randn(4, 784))
    # Test dropout in training vs inference
    dropout_test = Dropout(0.5)
    train_output = dropout_test.forward(test_x, training=True)
    infer_output = dropout_test.forward(test_x, training=False)
    assert np.array_equal(test_x.data, infer_output.data), "Inference mode should pass through unchanged"

    print("✅ Multi-layer network integration works!")

    print("\n" + "=" * 50)
    print("🎉 ALL TESTS PASSED! Module ready for export.")
    print("Run: tito module complete 03_layers")

if __name__=='__main__':
    test_module()
    

🧪 RUNNING MODULE INTEGRATION TEST
✅ Multi-layer network integration works!
Running unit tests...
🔬 Unit Test: Linear Layer...
✅ Linear layer works correctly!
🔬 Edge Case Tests: Linear Layer...
✅ Edge cases handled correctly!
🔬 Parameter Collection Test: Linear Layer...
✅ Parameter collection works correctly!
🔬 Unit Test: Dropout Layer...
✅ Dropout layer works correctly!

Running integration scenarios...
🔬 Integration Test: Multi-layer Network...
✅ Multi-layer network integration works!

🎉 ALL TESTS PASSED! Module ready for export.
Run: tito module complete 03_layers


In [10]:
if __name__ == "__main__":
    print("=" * 70)
    print("MODULE 03: LAYERS - COMPREHENSIVE VALIDATION")
    print("=" * 70)

    # Run module integration test
    test_module()

    print("\n" + "=" * 70)
    print("SYSTEMS ANALYSIS")
    print("=" * 70)

    # Run analysis functions
    analyze_layer_memory()
    print("\n")
    analyze_layer_performance()

    print("\n" + "=" * 70)
    print("✅ MODULE 03 COMPLETE!")
    print("=" * 70)

MODULE 03: LAYERS - COMPREHENSIVE VALIDATION
🧪 RUNNING MODULE INTEGRATION TEST
✅ Multi-layer network integration works!
Running unit tests...
🔬 Unit Test: Linear Layer...
✅ Linear layer works correctly!
🔬 Edge Case Tests: Linear Layer...
✅ Edge cases handled correctly!
🔬 Parameter Collection Test: Linear Layer...
✅ Parameter collection works correctly!
🔬 Unit Test: Dropout Layer...
✅ Dropout layer works correctly!

Running integration scenarios...
🔬 Integration Test: Multi-layer Network...
✅ Multi-layer network integration works!

🎉 ALL TESTS PASSED! Module ready for export.
Run: tito module complete 03_layers

SYSTEMS ANALYSIS
📊 Analyzing Layer Memory Usage...

Linear Layer Memory Analysis:
Configuration → Weight Memory → Bias Memory → Total Memory
( 784,  256) →   784.0 KB →    1.0 KB →   785.0 KB
( 256,  256) →   256.0 KB →    1.0 KB →   257.0 KB
( 256,   10) →    10.0 KB →    0.0 KB →    10.0 KB
(2048, 2048) → 16384.0 KB →    8.0 KB → 16392.0 KB

💡 Multi-layer Model Memory Scaling: